In [1]:
import pandas as pd
import nltk

In [2]:
df = pd.read_json('../../output_data/tc_nltk_preprocessed.json')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25763 entries, 0 to 25762
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            25763 non-null  object
 1   tag              25763 non-null  object
 2   s1_title_lower   25763 non-null  object
 3   s2_clean_title   25763 non-null  object
 4   s3_tokenized     25763 non-null  object
 5   s4_no_stopwords  25763 non-null  object
 6   s5_lemmatized    25763 non-null  object
dtypes: object(7)
memory usage: 1.4+ MB


In [4]:
df.head(4)

,title,tag,s1_title_lower,s2_clean_title,s3_tokenized,s4_no_stopwords,s5_lemmatized
0,My personal ranking of Community's use of alte...,COMMUNITY,my personal ranking of community's use of alte...,my personal ranking of communitys use of alter...,"[my, personal, ranking, of, communitys, use, o...","[personal, ranking, communitys, use, alternati...","[personal, ranking, community, use, alternativ..."
1,It took 30 years for climate tech investments ...,ENVIRONMENT,it took 30 years for climate tech investments ...,it took 30 years for climate tech investments ...,"[it, took, 30, years, for, climate, tech, inve...","[took, 30, years, climate, tech, investments, ...","[took, 30, year, climate, tech, investment, pa..."
2,Rain when sitting in a car,RELAX,rain when sitting in a car,rain when sitting in a car,"[rain, when, sitting, in, a, car]","[rain, sitting, car]","[rain, sitting, car]"
3,Is Cassis worth staying in?,TRAVEL,is cassis worth staying in?,is cassis worth staying in,"[is, cassis, worth, staying, in]","[cassis, worth, staying]","[cassis, worth, staying]"


In [5]:
# In order to perform padding, find max token count 

padded_df = pd.DataFrame({'lemmatized': df['s5_lemmatized']})

padded_df.head(5)

,lemmatized
0,"[personal, ranking, community, use, alternativ..."
1,"[took, 30, year, climate, tech, investment, pa..."
2,"[rain, sitting, car]"
3,"[cassis, worth, staying]"
4,"[new, research, suggests, black, hole, may, ac..."


In [6]:
padded_df['token_count'] = padded_df['lemmatized'].apply(lambda word_list: len(word_list))

padded_df.head(5)

,lemmatized,token_count
0,"[personal, ranking, community, use, alternativ...",6
1,"[took, 30, year, climate, tech, investment, pa...",13
2,"[rain, sitting, car]",3
3,"[cassis, worth, staying]",3
4,"[new, research, suggests, black, hole, may, ac...",10


In [7]:
import sys
 
sys.path.append('../')
from globals import SUPPORTED_NUMBER_OF_TOKENS

max_token_count = padded_df['token_count'].max()

print('max token count:', max_token_count)


# The max is 44, column padding will be adjusted to 50 (manually adjusted by the programmer) here I only print the set value
print('supported token count:', SUPPORTED_NUMBER_OF_TOKENS)

if SUPPORTED_NUMBER_OF_TOKENS < max_token_count:
    raise ValueError('SUPPORTED_NUMBER_OF_TOKENS is lower than max token count that exists in the dataset!')

max token count: 44
supported token count: 50


In [8]:
import numpy as np
from  tensorflow.keras.utils import pad_sequences

# Sample tokenized sequences
tokenized_sequences = [
    [1, 2, 3],
    [4, 5, 6, 7],
    [8, 9]
]

# Pad sequences and create masks
padded_sequences = pad_sequences(tokenized_sequences, padding='post')
masks = np.where(padded_sequences != 0, 1, 0)

print("Padded Sequences:")
print(padded_sequences)

print("\nMasks:")
print(masks)

import gensim.downloader as api
import nltk
from nltk.tokenize import word_tokenize

# Pobranie pre-trenowanego modelu Word2Vec
model = api.load("word2vec-google-news-300")

# Przykładowy lemmatized tekst
lemmatized_text = "new research suggests black hole may actually tangle fabric spacetime"

# Tokenizacja tekstu
tokens = word_tokenize(lemmatized_text)

# Konwersja tokenów na osadzenia słów (word embeddings)
word_embeddings = [model[token] for token in tokens if token in model]

print("Input Text:", lemmatized_text)
print("Word Embeddings:", word_embeddings)




2023-08-20 22:45:43.643045: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-20 22:45:43.663159: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-20 22:45:43.663637: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 22:45:44.288601: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Padded Sequences:
[[1 2 3 0]
 [4 5 6 7]
 [8 9 0 0]]

Masks:
[[1 1 1 0]
 [1 1 1 1]
 [1 1 0 0]]
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Input Text: new research suggests black hole may actually tangle fabric spacetime
Word Embeddings: [array([ 1.12915039e-02,  2.89306641e-02,  8.34960938e-02, -4.98046875e-02,
       -1.30859375e-01, -1.09863281e-01, -1.15234375e-01, -7.47070312e-02,
       -8.93554688e-02,  2.18750000e-01,  3.71093750e-02,  9.47265625e-02,
        2.34375000e-02, -8.44726562e-02, -2.65625000e-01, -1.00585938e-01,
        4.63867188e-03,  1.15234375e-01,  8.25195312e-02,  2.06298828e-02,
       -3.68652344e-02, -3.20434570e-04, -9.91210938e-02,  1.58691406e-02,
       -6.34765625e-02,  1.87500000e-01,  8.98437500e-02,  1.77734375e-01,
        1.20117188e-01,  3.71093750e-02, -1.19628906e-01,  4.05273438e-02,
       -3.22265625e-02, -3.80859375e-02,  1.26953125e-01, -1.58203125e-01,
       -5.07812500e-02, -1.56250000e-01,  8.